## Import Libraries

In [2]:
import sys
import os
from pathlib import Path
import torch
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
from utils import *
from configuration import *
import numpy as np
import pandas as pd

if not os.path.exists('./evaluation_date/'):
    os.makedirs('./evaluation_date/')

In [3]:
args = parser.parse_args([])
args.DEVICE = 'cpu'
args.task = 'temporal'
args.metric = 'temporal_acc'
args.SoftEva = True
args = FormulateArgs(args)

## Results on non-augmented test set

In [4]:
args.augment = False
args.augment

False

In [7]:
results = torch.zeros([15])

for ds in range(15):
        args.DATASET = ds
        seed = args.SEED
        test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')
        print(datainfo)

        modelname = f"pLF_data_{ds:02d}_{datainfo['dataname']}_seed_{seed:02d}.model"
        
        model_exist = os.path.isfile(f'./models/{modelname}')
        # aug_model_exist = os.path.isfile(f'../AugLearnableFilters/models/{modelname}')
        # baseline_model_exist = os.path.isfile(f'../LastBaseline/LearnableFilters/models/{modelname}')
        
        # print(model_exist and aug_model_exist and baseline_model_exist)
        # print(model_exist and aug_model_exist)
        
        
        # if model_exist and aug_model_exist:
        if model_exist:  
                          
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            # aug_model = torch.load(f'../AugLearnableFilters/models/{modelname}', map_location=args.DEVICE)
            # baseline_model = torch.load(f'../LastBaseline/LearnableFilters/models/{modelname}', map_location=baseline_args.DEVICE)
            
            model.UpdateArgs(args)
            # aug_model.UpdateArgs(args)
            # baseline_model.UpdateArgs(baseline_args)
            
            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)
            # baseline_evaluator = baselineEvaluator(baseline_args).to(baseline_args.DEVICE)

            for x,y in test_loader:
                X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)
                start_time = time.time()
                model(X_test[:1, :, :])
                end_time = time.time()
                inference_time = end_time - start_time
                results[ds] = inference_time
                break

        else:
            results[ds] = float('nan')

{'dataname': 'cbf', 'N_feature': 1, 'N_class': 3, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
torch.Size([100, 1, 64])
Inference Time for Batch 1: 4.6624109745025635 seconds
{'dataname': 'distalphalanxtw', 'N_feature': 1, 'N_class': 6, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
torch.Size([100, 1, 64])
Inference Time for Batch 1: 2.540848731994629 seconds
{'dataname': 'freezerregulartrain', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
torch.Size([100, 1, 64])
Inference Time for Batch 1: 2.123783588409424 seconds
{'dataname': 'freezersmalltrain', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
torch.Size([100, 1, 64])
Inference Time for Batch 1: 2.000450849533081 seconds
{'dataname': 'gunpointagespan', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 270, 'N_valid': 90, 'N_test': 91}
torch.Size([91, 1, 64])
Inference Time for Batch 1: 2.126366138458252 second

In [8]:
np.savetxt(f"./evaluation1/inf_time_2Order_LPF_acc.txt", results.numpy(), delimiter="\t", fmt='%.5f')

## Count devices

In [40]:
results = torch.zeros([15, 3])

for ds in range(15):
        args.DATASET = ds
        seed = args.SEED
        test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')
        print(datainfo)

        modelname = f"pLF_data_{ds:02d}_{datainfo['dataname']}_seed_{seed:02d}.model"
        
        model_exist = os.path.isfile(f'./models/{modelname}')
        # aug_model_exist = os.path.isfile(f'../AugLearnableFilters/models/{modelname}')
        # baseline_model_exist = os.path.isfile(f'../LastBaseline/LearnableFilters/models/{modelname}')
        
        # print(model_exist and aug_model_exist and baseline_model_exist)
        # print(model_exist and aug_model_exist)
        
        
        # if model_exist and aug_model_exist:
        if model_exist:  
                          
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            # aug_model = torch.load(f'../AugLearnableFilters/models/{modelname}', map_location=args.DEVICE)
            # baseline_model = torch.load(f'../LastBaseline/LearnableFilters/models/{modelname}', map_location=baseline_args.DEVICE)
            
            model.UpdateArgs(args)
            # aug_model.UpdateArgs(args)
            # baseline_model.UpdateArgs(baseline_args)
            
            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)
            # baseline_evaluator = baselineEvaluator(baseline_args).to(baseline_args.DEVICE)

            state_dict = model.state_dict()
            num_theta = 0
            num_r = 0
            num_c = 0
            for name in state_dict:
                if "theta_" in name:
                    num_theta += state_dict[name].shape[0] * state_dict[name].shape[1]
                elif "R_" in name:
                    num_r +=1
                elif "C_" in name:
                    num_c += 1
            
            results[ds][0] = num_theta
            results[ds][1] = num_r
            results[ds][2] = num_c

        else:
            results[ds] = float('nan')

{'dataname': 'cbf', 'N_feature': 1, 'N_class': 3, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
{'dataname': 'distalphalanxtw', 'N_feature': 1, 'N_class': 6, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
{'dataname': 'freezerregulartrain', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
{'dataname': 'freezersmalltrain', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
{'dataname': 'gunpointagespan', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 270, 'N_valid': 90, 'N_test': 91}
{'dataname': 'gunpointmaleversusfemale', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 270, 'N_valid': 90, 'N_test': 91}
{'dataname': 'gunpointoldversusyoung', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 270, 'N_valid': 90, 'N_test': 91}
{'dataname': 'middlephalanxoutlineagegroup', 'N_feature': 1, 'N_class': 3, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}

In [42]:
np.savetxt(f"./evaluation1/count_device_2Order_LPF_acc.txt", results.numpy(), delimiter="\t", fmt='%.5f')

In [6]:
results = torch.zeros([15,10,6])

for ds in range(15):
    args.DATASET = ds
    valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
    test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')
    print(datainfo)
    for seed in range(10):
        args.SEED = seed

        modelname = f"pLF_data_{ds:02d}_{datainfo['dataname']}_seed_{seed:02d}.model"
        
        model_exist = os.path.isfile(f'./models/{modelname}')
        # aug_model_exist = os.path.isfile(f'../AugLearnableFilters/models/{modelname}')
        # baseline_model_exist = os.path.isfile(f'../LastBaseline/LearnableFilters/models/{modelname}')
        
        # print(model_exist and aug_model_exist and baseline_model_exist)
        # print(model_exist and aug_model_exist)
        
        
        # if model_exist and aug_model_exist:
        if model_exist:  
                          
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            # aug_model = torch.load(f'../AugLearnableFilters/models/{modelname}', map_location=args.DEVICE)
            # baseline_model = torch.load(f'../LastBaseline/LearnableFilters/models/{modelname}', map_location=baseline_args.DEVICE)
            
            model.UpdateArgs(args)
            # aug_model.UpdateArgs(args)
            # baseline_model.UpdateArgs(baseline_args)
            
            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)
            # baseline_evaluator = baselineEvaluator(baseline_args).to(baseline_args.DEVICE)

            for x,y in valid_loader:
                X_valid, y_valid = x.to(args.DEVICE), y.to(args.DEVICE)
            for x,y in test_loader:
                X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)

            acc_valid = evaluator(model, X_valid, y_valid)
            acc_test   = evaluator(model, X_test,  y_test)
            
            # aug_acc_valid = evaluator(aug_model, X_valid, y_valid)
            # aug_acc_test   = evaluator(aug_model, X_test,  y_test)
            
            # baseline_acc_valid = baseline_evaluator(baseline_model, X_valid, y_valid)
            # baseline_acc_test   = baseline_evaluator(baseline_model, X_test,  y_test)

            results[ds,seed,0] = acc_valid
            results[ds,seed,1] = acc_test
            
            # results[ds,seed,2] = aug_acc_valid
            # results[ds,seed,3] = aug_acc_test
            
            # results[ds,seed,4] = baseline_acc_valid
            # results[ds,seed,5] = baseline_acc_test
        else:
            results[ds,seed,:] = float('nan')

{'dataname': 'cbf', 'N_feature': 1, 'N_class': 3, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
{'dataname': 'distalphalanxtw', 'N_feature': 1, 'N_class': 6, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
{'dataname': 'freezerregulartrain', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
{'dataname': 'freezersmalltrain', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
{'dataname': 'gunpointagespan', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 270, 'N_valid': 90, 'N_test': 91}
{'dataname': 'gunpointmaleversusfemale', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 270, 'N_valid': 90, 'N_test': 91}
{'dataname': 'gunpointoldversusyoung', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 270, 'N_valid': 90, 'N_test': 91}
{'dataname': 'middlephalanxoutlineagegroup', 'N_feature': 1, 'N_class': 3, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}

In [7]:
N_selected_seeds = 3
re_temp = torch.nan_to_num(results, nan=-10000.)
values, indices = torch.topk(re_temp[:,:,1], k=N_selected_seeds, dim=1)
mean_selected = torch.mean(torch.asarray(values), dim=1)
var_selected = torch.std(torch.asarray(values), dim=1)
selected_results = torch.cat([mean_selected.unsqueeze(-1), var_selected.unsqueeze(-1)], dim=1)

np.savetxt(f"./evaluation1/non_var_test_top_{N_selected_seeds}_2Order_LPF_acc.txt", selected_results.numpy(), delimiter="\t", fmt='%.5f')

In [44]:
N_selected_seeds = 3
re_temp = torch.nan_to_num(results, nan=-10000.)
values, indices = torch.topk(re_temp[:,:,3], k=N_selected_seeds, dim=1)
mean_selected = torch.mean(torch.asarray(values), dim=1)
var_selected = torch.std(torch.asarray(values), dim=1)
selected_results = torch.cat([mean_selected.unsqueeze(-1), var_selected.unsqueeze(-1)], dim=1)

np.savetxt(f"./evaluation1/non_var_test_top_{N_selected_seeds}_aug_LPF_acc.txt", selected_results.numpy(), delimiter="\t", fmt='%.5f')

## Results on augmented test set

In [4]:
args.augment = True
args.augment

True

## Proposed model

In [6]:
import PowerPrintedLearnableFilter as pNN
args.augment = True
all_results = []
results = torch.zeros([15,10,6])
for ds in range(15):
    args.DATASET = ds
    for seed in range(10):
        args.SEED = seed
        valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
        modelname = f"pLF_data_{ds:02d}_{datainfo['dataname']}_seed_{seed:02d}.model"
        model_exist = os.path.isfile(f'./models/{modelname}')
        print(model_exist)
        pnn = pNN.PrintedNeuralNetwork(args, datainfo['N_feature'], datainfo['N_class'], args.N_Channel, N_feature=args.N_feature).to(args.DEVICE)
        if model_exist:  
                          
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            pnn.load_state_dict(model.state_dict())
            pnn.UpdateArgs(args)
            
            print(pnn.MAC_power)

            # SetSeed(args.SEED)

#             evaluator = Evaluator(args).to(args.DEVICE)
#             # baseline_evaluator = baselineEvaluator(baseline_args).to(baseline_args.DEVICE)

#             for x,y in valid_loader:
#                 X_valid, y_valid = x.to(args.DEVICE), y.to(args.DEVICE)
#             for x,y in test_loader:
#                 X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)

#             acc_valid = evaluator(model, X_valid, y_valid)
#             acc_test   = evaluator(model, X_test,  y_test)

#             results[ds,seed,0] = acc_valid
#             results[ds,seed,1] = acc_test
            
#             inference_time = 0
            
#             for x,y in test_loader:
#                 X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)
#                 start_time = time.time()
#                 model(X_test[:1, :, :])
#                 end_time = time.time()
#                 inference_time = end_time - start_time
#                 break
#             results[ds, seed, 2] = inference_time
            
#             # results[ds,seed,3] = area_test
#             # results[ds,seed,4] = power_test
            
#         else:
#             results[ds,seed,:] = float('nan')
            
#         temp_result = [datainfo['dataname'], seed, results[ds, seed, 0].item(), results[ds, seed, 1].item(), results[ds, seed, 2].item()]
#         all_results.append(temp_result)

# columns = ['dataset', 'seed', 'acc_valid', 'acc_test', 'average_time']
# all_results.sort(key=lambda x: x[0])            
# df = pd.DataFrame(all_results, columns=columns)
# # Save the DataFrame to an Excel file
# if not os.path.exists(f"./evaluation2_date/"):
#     os.makedirs(f"./evaluation2_date/")
# excel_filename = f"./evaluation2_date/augment_{args.augment}_proposed_evaluation_results_analysis.xlsx"
# df.to_excel(excel_filename, index=False)
# print(f"Results have been saved to {excel_filename}")

False


FileNotFoundError: [Errno 2] No such file or directory: './utils/act_model_package'

In [8]:
args.augment = True
all_results = []
results = torch.zeros([15,10,6])
for ds in range(15):
    args.DATASET = ds
    valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
    test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')  
    print(datainfo)
    for seed in range(10):
        args.SEED = seed


        modelname = f"pLF_data_{ds:02d}_{datainfo['dataname']}_seed_{seed:02d}.model"
        
        model_exist = os.path.isfile(f'./models/{modelname}')

        print(model_exist)
        
        
        if model_exist:  
                          
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            model.UpdateArgs(args)

            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)
            # baseline_evaluator = baselineEvaluator(baseline_args).to(baseline_args.DEVICE)

            for x,y in valid_loader:
                X_valid, y_valid = x.to(args.DEVICE), y.to(args.DEVICE)
            for x,y in test_loader:
                X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)

            acc_valid = evaluator(model, X_valid, y_valid)
            acc_test   = evaluator(model, X_test,  y_test)

            results[ds,seed,0] = acc_valid
            results[ds,seed,1] = acc_test
            
            inference_time = 0
            
            for x,y in test_loader:
                X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)
                start_time = time.time()
                model(X_test[:1, :, :])
                end_time = time.time()
                inference_time = end_time - start_time
                break
            results[ds, seed, 2] = inference_time
            
            # results[ds,seed,3] = area_test
            # results[ds,seed,4] = power_test
            
        else:
            results[ds,seed,:] = float('nan')
            
        temp_result = [datainfo['dataname'], seed, results[ds, seed, 0].item(), results[ds, seed, 1].item(), results[ds, seed, 2].item()]
        all_results.append(temp_result)

columns = ['dataset', 'seed', 'acc_valid', 'acc_test', 'average_time']
all_results.sort(key=lambda x: x[0])            
df = pd.DataFrame(all_results, columns=columns)
# Save the DataFrame to an Excel file
if not os.path.exists(f"./evaluation2_date/"):
    os.makedirs(f"./evaluation2_date/")
excel_filename = f"./evaluation2_date/augment_{args.augment}_proposed_evaluation_results_analysis.xlsx"
df.to_excel(excel_filename, index=False)
print(f"Results have been saved to {excel_filename}")

{'dataname': 'cbf', 'N_feature': 1, 'N_class': 3, 'N_time': 64, 'N_train': 600, 'N_valid': 200, 'N_test': 200}
True
True
True
True
True
True
True
True
True
True
{'dataname': 'distalphalanxtw', 'N_feature': 1, 'N_class': 6, 'N_time': 64, 'N_train': 600, 'N_valid': 200, 'N_test': 200}
True
True
True
True
True
True
True
True
True
True
{'dataname': 'freezerregulartrain', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 600, 'N_valid': 200, 'N_test': 200}
True
True
True
True
True
True
True
True
True
True
{'dataname': 'freezersmalltrain', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 600, 'N_valid': 200, 'N_test': 200}
True
True
True
True
True
True
True
True
True
True
{'dataname': 'gunpointagespan', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 540, 'N_valid': 180, 'N_test': 182}
True
True
True
True
True
True
True
True
True
True
{'dataname': 'gunpointmaleversusfemale', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 540, 'N_valid': 180, 'N_test': 182}
True
True

In [37]:
N_selected_seeds = 3
re_temp = torch.nan_to_num(results, nan=-10000.)
values, indices = torch.topk(re_temp[:,:,1], k=N_selected_seeds, dim=1)
mean_selected = torch.mean(torch.asarray(values), dim=1)
var_selected = torch.std(torch.asarray(values), dim=1)
selected_results = torch.cat([mean_selected.unsqueeze(-1), var_selected.unsqueeze(-1)], dim=1)

np.savetxt(f"./evaluation1/var_test_top_{N_selected_seeds}_var_LPF_acc.txt", selected_results.numpy(), delimiter="\t", fmt='%.5f')

In [38]:
N_selected_seeds = 3
re_temp = torch.nan_to_num(results, nan=-10000.)
values, indices = torch.topk(re_temp[:,:,3], k=N_selected_seeds, dim=1)
mean_selected = torch.mean(torch.asarray(values), dim=1)
var_selected = torch.std(torch.asarray(values), dim=1)
selected_results = torch.cat([mean_selected.unsqueeze(-1), var_selected.unsqueeze(-1)], dim=1)

np.savetxt(f"./evaluation1/var_test_top_{N_selected_seeds}_aug_LPF_acc.txt", selected_results.numpy(), delimiter="\t", fmt='%.5f')

In [ ]:
args.augment = True
all_results = []
results = torch.zeros([15,10,6])
for ds in range(9):
    args.DATASET = ds
    valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
    test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')  
    print(datainfo)
    for seed in range(10):
        args.SEED = seed


        modelname = f"pLF_data_{ds:02d}_{datainfo['dataname']}_seed_{seed:02d}.model"
        
        model_exist = os.path.isfile(f'./models/{modelname}')
        # aug_model_exist = os.path.isfile(f'../AugLearnableFilters/models/{modelname}')
        # baseline_model_exist = os.path.isfile(f'../LastBaseline/LearnableFilters/models/{modelname}')
        
        # print(model_exist and aug_model_exist and baseline_model_exist)
        print(model_exist)
        
        
        if model_exist:  
                          
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            # aug_model = torch.load(f'../AugLearnableFilters/models/{modelname}', map_location=args.DEVICE)
            # baseline_model = torch.load(f'../LastBaseline/LearnableFilters/models/{modelname}', map_location=baseline_args.DEVICE)
            
            model.UpdateArgs(args)
            # aug_model.UpdateArgs(args)
            # baseline_model.UpdateArgs(baseline_args)
            
            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)
            # baseline_evaluator = baselineEvaluator(baseline_args).to(baseline_args.DEVICE)

            for x,y in valid_loader:
                X_valid, y_valid = x.to(args.DEVICE), y.to(args.DEVICE)
            for x,y in test_loader:
                X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)

            acc_valid = evaluator(model, X_valid, y_valid)
            acc_test   = evaluator(model, X_test,  y_test)
            
            # aug_acc_valid = evaluator(aug_model, X_valid, y_valid)
            # aug_acc_test   = evaluator(aug_model, X_test,  y_test)
            
            # baseline_acc_valid = baseline_evaluator(baseline_model, X_valid, y_valid)
            # baseline_acc_test   = baseline_evaluator(baseline_model, X_test,  y_test)

            results[ds,seed,0] = acc_valid
            results[ds,seed,1] = acc_test
            
            # results[ds,seed,2] = aug_acc_valid
            # results[ds,seed,3] = aug_acc_test
            
            # results[ds,seed,4] = baseline_acc_valid
            # results[ds,seed,5] = baseline_acc_test
        else:
            results[ds,seed,:] = float('nan')
            
        inference_time = 0
            
        for x,y in test_loader:
            X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)
            start_time = time.time()
            model(X_test[:1, :, :])
            end_time = time.time()
            inference_time = end_time - start_time
            break
        results[ds, seed, 2] = inference_time
            
        temp_result = [datainfo['dataname'], seed, results[ds, seed, 0].item(), results[ds, seed, 1].item(), results[ds, seed, 2].item()]
        all_results.append(temp_result)

columns = ['dataset', 'seed', 'acc_valid', 'acc_test', 'average_time']
all_results.sort(key=lambda x: x[0])            
df = pd.DataFrame(all_results, columns=columns)
# Save the DataFrame to an Excel file
if not os.path.exists(f"./evaluation/"):
    os.makedirs(f"./evaluation/")
excel_filename = f"./evaluation/ELMAN_evaluation_results_analysis.xlsx"
df.to_excel(excel_filename, index=False)
print(f"Results have been saved to {excel_filename}")